In [27]:
import json
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(json.dumps(documents[0], indent=2))

In [ ]:
from elasticsearch import Elasticsearch
from tqdm import tqdm

es_client = Elasticsearch('http://localhost:9200') 

print(es_client.info())

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
   es_client.index(index=index_name, body=doc)

In [ ]:
def elastic_search_index(query):
    es_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=es_query)
    return response

print(elastic_search_index("How do execute a command on a Kubernetes pod?"))

In [59]:
query = "How do copy a file to a Docker container?"

es_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
response = es_client.search(index=index_name, body=es_query)
# print(response)

context_list = []
context_template = """
Q: {question}
A: {text}
""".strip()

for hit in response["hits"]["hits"]:
    temp_res = hit['_source']
    context_list.append(context_template.format(question=temp_res['question'], text=temp_res['text']))

context = "\n\n".join(context_list)
# print(context)


In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question="How do copy a file to a Docker container?", context=context)

print(len(prompt))

In [ ]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

print(len(prompt))
output = encoding.encode(prompt)
# print(output)
print(len(output))